In [1]:
from datascience import *
from datascience.predicates import are
path_data = '../../../../data/'
import numpy as np
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

from urllib.request import urlopen
import re
from bancor_research.bancor_simulator.v3.spec.network import BancorDapp

# Recall from an earlier chapter that we already defined the whitelisted_tokens as follows.
whitelisted_tokens = {
    "eth": {
        "trading_fee": "1%",
        "bnt_funding_limit": "1000000"
    },
    "link": {
        "trading_fee": "1%",
        "bnt_funding_limit": "1000000"
    },
    "wbtc": {
        "trading_fee": "1%",
        "bnt_funding_limit": "1000000"
    }
}
# There are other possible configuration settings available, however for the present purpose we will use the defaults.
v3 = BancorDapp(whitelisted_tokens=whitelisted_tokens)

v3.create_user('Trader')
v3.create_user('David')

v3.set_user_balance(tkn_amt='101', tkn_name='eth', user_name='Alice')
v3.set_user_balance(tkn_amt='101', tkn_name='wbtc', user_name='Bob')
v3.set_user_balance(tkn_amt='10001', tkn_name='link', user_name='Charlie')
v3.set_user_balance(tkn_amt='2000', tkn_name='bnt', user_name='Trader')
v3.set_user_balance(tkn_amt='1', tkn_name='eth', user_name='Trader')
v3.set_user_balance(tkn_amt='3', tkn_name='link', user_name='Trader')
v3.set_user_balance(tkn_amt='1000', tkn_name='bnt', user_name='David')

# The actual deposits.
v3.deposit(tkn_amt='100', tkn_name='eth', user_name='Alice')

v3.deposit(tkn_amt='10000', tkn_name='link', user_name='Charlie')

v3.deposit(tkn_amt='100', tkn_name='wbtc', user_name='Bob')

v3.deposit(tkn_amt='1', tkn_name='eth', user_name='Alice')

v3.deposit(tkn_amt='1', tkn_name='link', user_name='Charlie')

v3.deposit(tkn_amt='1', tkn_name='wbtc', user_name='Bob')

for tkn_name in whitelisted_tokens:
    v3.enable_trading(tkn_name)

v3.trade(tkn_amt='2000', source_token='bnt',  target_token='link', user_name='Trader', timestamp=0)

v3.trade(tkn_amt='302.9981', source_token='link',  target_token='bnt', user_name='Trader', timestamp=0)

v3.trade(tkn_amt='1', source_token='eth',  target_token='wbtc', user_name='Trader', timestamp=0)

def read_url(url): 
    return re.sub('\\s+', ' ', urlopen(url).read().decode())

Bootstrap requirements met for eth
Bootstrap requirements met for link
Bootstrap requirements met for wbtc


# Staking BNT

In Bancor 3, assume that BNT liquidity providers are receiving protocol bnBNT tokens in return for destroying BNT. The calculation is essentially identical to the standard pool token method, and its outcome is relatively easy to understand. However, there are some nuances to the process that should be highlighted.

For this section, we can consider the state of the system after the initial bootstrapping and first trading activity described earlier. The system snapshot is as follows:


In [2]:
v3.describe(decimals=4)

eth        link        wbtc  \
1 Account Alice                         0.0000         NaN         NaN   
          Bob                              NaN         NaN      0.0000   
          Charlie                          NaN      0.0000         NaN   
          David                            NaN         NaN         NaN   
          Trader                        0.0000      0.0019      0.0557   
2 Pool    a: TKN Staked Balance       101.0000  10003.4242    101.0005   
          b: TKN Trading Liquidity     21.0000   3336.3314      1.1943   
          c: BNT Trading Liquidity  19055.2381  20013.9939  20940.8835   
          d: BNT Current Funding    20007.6190  20015.9807  20000.0000   
          e: Spot Rate                907.3923      5.9988  17534.2111   
          f: Average Rate            1000.0000      6.0000  16000.0000   
          g: Average Inverse Rate       0.0000      0.0000      0.0000   
3 Network Master Vault                102.0000  10003.9981    100.9443   
          Protection Vault              0.0000      0.0000      0.0000   
          Protocol Equity                  NaN         NaN         NaN   
          Rewards Vault                    NaN         NaN         NaN   

                                          bnt     bneth      bnlink    bnwbtc  
1 Account Alice                           NaN  101.0000         NaN       NaN  
          Bob                             NaN       NaN         NaN  101.0000  
          Charlie                         NaN       NaN  10001.0000       NaN  
          David                     1000.0000       NaN         NaN       NaN  
          Trader                    1977.6153       NaN         NaN       NaN  
2 Pool    a: TKN Staked Balance           NaN    0.0000      0.0000    0.0000  
          b: TKN Trading Liquidity        NaN    0.0000      0.0000    0.0000  
          c: BNT Trading Liquidity        NaN    0.0000      0.0000    0.0000  
          d: BNT Current Funding          NaN    0.0000      0.0000    0.0000  
          e: Spot Rate                    NaN    0.0000      0.0000    0.0000  
          f: Average Rate                 NaN    0.0000      0.0000    0.0000  
          g: Average Inverse Rate         NaN    0.0000      0.0000    0.0000  
3 Network Master Vault                    NaN       NaN         NaN       NaN  
          Protection Vault                NaN       NaN         NaN       NaN  
          Protocol Equity                 NaN    0.0000      0.0000    0.0000  
          Rewards Vault                   NaN    0.0000      0.0000    0.0000

In [3]:
timestamp = 0
v3.deposit(tkn_amt='1000', tkn_name='bnt',  user_name='David', timestamp=timestamp)

v3.describe(decimals=4)

eth        link        wbtc  \
1 Account Alice                         0.0000         NaN         NaN   
          Bob                              NaN         NaN      0.0000   
          Charlie                          NaN      0.0000         NaN   
          David                            NaN         NaN         NaN   
          Trader                        0.0000      0.0019      0.0557   
2 Pool    a: TKN Staked Balance       101.0000  10003.4242    101.0005   
          b: TKN Trading Liquidity     21.0000   3336.3314      1.1943   
          c: BNT Trading Liquidity  19055.2381  20013.9939  20940.8835   
          d: BNT Current Funding    20007.6190  20015.9807  20000.0000   
          e: Spot Rate                907.3923      5.9988  17534.2111   
          f: Average Rate            1000.0000      6.0000  16000.0000   
          g: Average Inverse Rate       0.0000      0.0000      0.0000   
3 Network Master Vault                102.0000  10003.9981    100.9443   
          Protection Vault              0.0000      0.0000      0.0000   
          Protocol Equity                  NaN         NaN         NaN   
          Rewards Vault                    NaN         NaN         NaN   

                                          bnt     bneth      bnlink    bnwbtc  
1 Account Alice                           NaN  101.0000         NaN       NaN  
          Bob                             NaN       NaN         NaN  101.0000  
          Charlie                         NaN       NaN  10001.0000       NaN  
          David                        0.0000       NaN         NaN       NaN  
          Trader                    1977.6153       NaN         NaN       NaN  
2 Pool    a: TKN Staked Balance           NaN    0.0000      0.0000    0.0000  
          b: TKN Trading Liquidity        NaN    0.0000      0.0000    0.0000  
          c: BNT Trading Liquidity        NaN    0.0000      0.0000    0.0000  
          d: BNT Current Funding          NaN    0.0000      0.0000    0.0000  
          e: Spot Rate                    NaN    0.0000      0.0000    0.0000  
          f: Average Rate                 NaN    0.0000      0.0000    0.0000  
          g: Average Inverse Rate         NaN    0.0000      0.0000    0.0000  
3 Network Master Vault                    NaN       NaN         NaN       NaN  
          Protection Vault                NaN       NaN         NaN       NaN  
          Protocol Equity                 NaN    0.0000      0.0000    0.0000  
          Rewards Vault                   NaN    0.0000      0.0000    0.0000

The most important nuance to be aware of is that BNT liquidity providers are supporting the BNT liquidity of the whole protocol, rather than any specific pool. As a result, there is no spot price or moving average. Further, bnBNT pool tokens are not created when a BNT liquidity provider adds their tokens to the protocol; there is no change in the vault balance of BNT, or its staked balance (save for one extreme edge case). The provision of BNT by users is best thought of as a private exchange of BNT directly for protocol-owned bnBNT.

To demonstrate, assume a fourth participant, David, wishes to provide 1,000 BNT liquidity to the protocol. The only calculations the protocol must perform are to value the BNT David is providing. The staking ledger is reporting a total of 6,002.3599 BNT, and the bnBNT pool token supply is 6,000 bnBNT. Therefore, the bnBNT/BNT exchange rate is 0.9996068, and David’s 1,000 BNT is worth 999.60683797 bnBNT. When David confirms this transaction, the protocol simply transfers this amount of bnBNT to him, from its own balance; the BNT that David provided is burned immediately.

Note that the only observable change in the system is that the protocol owns less of the total BNT; outside of the system, the ERC20 contract supply of BNT will have diminished slightly. David also receives vBNT, the Bancor governance token, at a 1:1 rate with respect to the bnBNT pool tokens (see the following section). vBNT is staked in the governance contract to take part in Bancor’s decision making process; however, it must also be returned when a user exits from the protocol.

In [4]:
v3.describe(decimals=4)

eth        link        wbtc  \
1 Account Alice                         0.0000         NaN         NaN   
          Bob                              NaN         NaN      0.0000   
          Charlie                          NaN      0.0000         NaN   
          David                            NaN         NaN         NaN   
          Trader                        0.0000      0.0019      0.0557   
2 Pool    a: TKN Staked Balance       101.0000  10003.4242    101.0005   
          b: TKN Trading Liquidity     21.0000   3336.3314      1.1943   
          c: BNT Trading Liquidity  19055.2381  20013.9939  20940.8835   
          d: BNT Current Funding    20007.6190  20015.9807  20000.0000   
          e: Spot Rate                907.3923      5.9988  17534.2111   
          f: Average Rate            1000.0000      6.0000  16000.0000   
          g: Average Inverse Rate       0.0000      0.0000      0.0000   
3 Network Master Vault                102.0000  10003.9981    100.9443   
          Protection Vault              0.0000      0.0000      0.0000   
          Protocol Equity                  NaN         NaN         NaN   
          Rewards Vault                    NaN         NaN         NaN   

                                          bnt     bneth      bnlink    bnwbtc  
1 Account Alice                           NaN  101.0000         NaN       NaN  
          Bob                             NaN       NaN         NaN  101.0000  
          Charlie                         NaN       NaN  10001.0000       NaN  
          David                        0.0000       NaN         NaN       NaN  
          Trader                    1977.6153       NaN         NaN       NaN  
2 Pool    a: TKN Staked Balance           NaN    0.0000      0.0000    0.0000  
          b: TKN Trading Liquidity        NaN    0.0000      0.0000    0.0000  
          c: BNT Trading Liquidity        NaN    0.0000      0.0000    0.0000  
          d: BNT Current Funding          NaN    0.0000      0.0000    0.0000  
          e: Spot Rate                    NaN    0.0000      0.0000    0.0000  
          f: Average Rate                 NaN    0.0000      0.0000    0.0000  
          g: Average Inverse Rate         NaN    0.0000      0.0000    0.0000  
3 Network Master Vault                    NaN       NaN         NaN       NaN  
          Protection Vault                NaN       NaN         NaN       NaN  
          Protocol Equity                 NaN    0.0000      0.0000    0.0000  
          Rewards Vault                   NaN    0.0000      0.0000    0.0000